In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

#import time library and the datetime module
import time
from datetime import datetime

#import requests and api key
import requests
from config import weather_api_key

In [2]:
#create a set  of random lat and long combos
#-------------------------CHANGE SIZE TO 1500 BEFORE SUBMISSION----------------------------
lats = np.random.uniform(low=-90.000,high=90.000,size=100)
lngs = np.random.uniform(low=-90.000,high=90.000,size=100)
lat_lngs = zip(lats,lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)

In [4]:
#get lat and long combo in sets
for coordinate in coordinates:
    print(coordinate[0],coordinate[1])

86.96756317640566 20.190241227353724
61.003128207023764 -71.64361804648227
-59.37983740945012 65.58699849156099
22.79793966019946 89.90897322208198
77.40323612638531 1.9822424439386026
-49.58297017563469 75.7454610396126
-54.83340673352729 -24.993747502822046
32.15127903980128 -23.219817888708036
-55.112873628628556 52.04709245010366
35.48808086440752 -9.89643918362303
-16.505879005010655 65.95544182025776
-51.961044661688106 6.713910326069097
78.39831349694447 2.204508257347584
-70.79365162415041 -16.582345708536423
83.10155020406373 22.75225282439787
80.03737460412643 6.297191681331114
-30.13153917484594 33.65309993569207
18.009333164170485 42.432659727968485
-86.82168718443619 -53.74566369049855
-81.99693021565255 28.181911972994527
-32.93428207439343 -61.25465723682885
48.588926910209665 35.38968597864262
21.14486038660185 -55.40108855795059
-25.267755783359775 -44.64022864890891
16.51235045443711 32.00650871223205
-6.724100956041639 53.08193546707827
-72.50938354638893 86.34401106

In [5]:
#create list for holding the cities
cities = []

#identify nearest city for each lat and long combo

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    #if city is unique, add to list
    if city not in cities:
        cities.append(city)
        
#print city count and confirm sufficiency
len(cities)

71

In [6]:
#build the url
url = "http://api.openweathermap.org/data/2.5/weather?units=metric&appid=" + weather_api_key

#create list for holding the cities
city_data = []

#print beginning of logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

#create counters
record_count = 1
set_count = 1

#NOTA BENE: we need to iterate through our list of cities and begin building the URL for each city, while grouping our records in sets of 50.
#o do this, use for i in range(len(cities)) and the index to tell us when we get to 50. Once we get to 50, we tell the program to pause for 60 seconds using the time.sleep(60) command. The OpenWeatherMap API only allows 60 calls for per minute on their free tier, so pausing our program for one minute after each set of 50 will prevent time-out errors. We can also retrieve the city from the cities list and add it to the city_url by using indexing,
#as shown in the following code:

#loop through all cities in our list
for i,city in enumerate (cities):
    #group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count +=1
        record_count = 1
        time.sleep(60)
        
    #create endpoint url for each city
    city_url = url + "&q=" + city.replace(" ", "+")
    
    #log the url, record and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    #add 1 to record count
    record_count +=1

    #run api request for each city
    try:
        #parse json and retrieve data
        city_weather = requests.get(city_url).json()

        #parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]

        #convert data in ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        #append city data to city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    #if error occur, skip city
    except:
        print("City not found. Skipping...")
        pass

#indicate data loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | longyearbyen
Processing Record 2 of Set 1 | iqaluit
Processing Record 3 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 4 of Set 1 | pirojpur
Processing Record 5 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 6 of Set 1 | mahebourg
Processing Record 7 of Set 1 | cidreira
Processing Record 8 of Set 1 | vila franca do campo
Processing Record 9 of Set 1 | lagos
Processing Record 10 of Set 1 | quatre cocos
Processing Record 11 of Set 1 | cape town
Processing Record 12 of Set 1 | richards bay
Processing Record 13 of Set 1 | abha
Processing Record 14 of Set 1 | ushuaia
Processing Record 15 of Set 1 | kruisfontein
Processing Record 16 of Set 1 | rosario
Processing Record 17 of Set 1 | pidhorodne
Processing Record 18 of Set 1 | codrington
Processing Record 19 of Set 1 | ilhabela
Processing Record 20 of Set 1 | umm durman
City not found. Skipping...
Processing Reco

In [7]:
len(city_data)

62

In [8]:
#convert array of dictionaries to Pandas DF
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Longyearbyen,78.2186,15.6401,2.91,87,75,2.06,SJ,2022-10-02 19:20:01
1,Iqaluit,63.7506,-68.5145,0.85,64,75,6.71,CA,2022-10-02 19:18:52
2,Pirojpur,22.5667,89.9833,25.25,95,100,2.88,BD,2022-10-02 19:20:01
3,Mahebourg,-20.4081,57.7000,22.16,68,75,4.12,MU,2022-10-02 19:20:02
4,Cidreira,-30.1811,-50.2056,18.17,79,74,3.01,BR,2022-10-02 19:16:44
5,Vila Franca Do Campo,37.7167,-25.4333,24.55,87,72,6.37,PT,2022-10-02 19:20:02
6,Lagos,6.5833,3.7500,27.18,89,100,0.43,NG,2022-10-02 19:15:05
7,Quatre Cocos,-20.2078,57.7625,22.06,71,95,6.46,MU,2022-10-02 19:20:03
8,Cape Town,-33.9258,18.4232,16.60,77,0,2.24,ZA,2022-10-02 19:20:03
9,Richards Bay,-28.7830,32.0377,20.85,94,4,2.01,ZA,2022-10-02 19:17:04


In [9]:
#re-arrange the column headers
new_col_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_col_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Longyearbyen,SJ,2022-10-02 19:20:01,78.2186,15.6401,2.91,87,75,2.06
1,Iqaluit,CA,2022-10-02 19:18:52,63.7506,-68.5145,0.85,64,75,6.71
2,Pirojpur,BD,2022-10-02 19:20:01,22.5667,89.9833,25.25,95,100,2.88
3,Mahebourg,MU,2022-10-02 19:20:02,-20.4081,57.7000,22.16,68,75,4.12
4,Cidreira,BR,2022-10-02 19:16:44,-30.1811,-50.2056,18.17,79,74,3.01
5,Vila Franca Do Campo,PT,2022-10-02 19:20:02,37.7167,-25.4333,24.55,87,72,6.37
6,Lagos,NG,2022-10-02 19:15:05,6.5833,3.7500,27.18,89,100,0.43
7,Quatre Cocos,MU,2022-10-02 19:20:03,-20.2078,57.7625,22.06,71,95,6.46
8,Cape Town,ZA,2022-10-02 19:20:03,-33.9258,18.4232,16.60,77,0,2.24
9,Richards Bay,ZA,2022-10-02 19:17:04,-28.7830,32.0377,20.85,94,4,2.01


In [11]:
#create the output csv file
output_data_file = "weather_data\cities.csv"

#export the city_data into the csv
city_data_df.to_csv(output_data_file, index_label = "City_ID")